In [18]:
%pip install pandas

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [21]:
import pandas as pd
import re

def clean_users(input_file):
    df = pd.read_csv(input_file)
    
    print("Original data shape:", df.shape)
    
    df = df.drop_duplicates(subset='user_id', keep='last')
    print("After removing duplicates:", df.shape)

    email_regex = r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$'
    
    df['email'] = df['email'].str.replace('@@', '@', regex=False)
    df['email'] = df['email'].str.replace('..com', '.com', regex=False)
    df['email'] = df['email'].str.replace(' ', '')

    valid_emails = df['email'].apply(lambda x: bool(re.match(email_regex, str(x))))
    invalid_email_count = len(df[~valid_emails])
    print(f"Removing {invalid_email_count} invalid email entries")
    df = df[valid_emails].copy()

    age_mask = df['age'].between(13, 100)
    invalid_age_count = len(df[~age_mask])
    print(f"Removing {invalid_age_count} invalid age entries")
    df = df[age_mask].copy()

    country_mapping = {
        'US': 'United States',
        'USA': 'United States',
        'UK': 'United Kingdom',
        'NULL': 'Unknown',
        'XYZ': 'Unknown'
    }
    
    df['country'] = (
        df['country']
        .str.strip()
        .replace(country_mapping)
        .fillna('Unknown')
        .apply(lambda x: x if x in df['country'].unique() else 'Unknown')
    )

    df['date'] = pd.to_datetime(df['date'], errors='coerce')
    df = df[df['date'].notna()].copy()

    df['name'] = (
        df['name']
        .str.replace(r'\s+', ' ', regex=True)
        .str.replace(r'[^\w\s.]', '', regex=True)
        .str.title()
    )

    df = df.dropna(subset=['email', 'country']).reset_index(drop=True)
    
    print("Final cleaned data shape:", df.shape)
    print("\nSample cleaned data:")
    print(df.head())
    
    return df

cleaned_users = clean_users("users.csv")
cleaned_users.to_csv("cleaned_users.csv", index=False)

validation_report = {
    'original_entries': 165,
    'final_entries': len(cleaned_users),
    'duplicates_removed': 165 - len(cleaned_users)
}

print("\nValidation Report:")
print(f"Original entries: {validation_report['original_entries']}")
print(f"Final valid entries: {validation_report['final_entries']}")
print(f"Invalid entries removed: {validation_report['original_entries'] - validation_report['final_entries']}")


Original data shape: (2051, 6)
After removing duplicates: (2001, 6)
Removing 36 invalid email entries
Removing 23 invalid age entries
Final cleaned data shape: (1942, 6)

Sample cleaned data:
   user_id             name                         email  age  \
0    15000      James Scott      james.scott660@gmail.com   35   
1    15001   Sharon Ferrell    sharon.ferrell49@gmail.com   26   
2    15002    Chelsea Ortiz    chelsea.ortiz643@gmail.com   38   
3    15003  Gregory Pearson  gregory.pearson379@gmail.com   52   
4    15004  Connie Gonzales  connie.gonzales481@gmail.com   43   

          country       date  
0           India 2024-11-01  
1       Australia 2024-11-01  
2  United Kingdom 2024-11-01  
3  United Kingdom 2024-11-01  
4       Australia 2024-11-01  

Validation Report:
Original entries: 165
Final valid entries: 1942
Invalid entries removed: -1777


In [19]:
import pandas as pd

subscriptions = pd.read_csv("subscriptions.csv")

# Remove duplicates
subscriptions = subscriptions.drop_duplicates(subset='payment_id')

# Fix incorrect amounts
def correct_amount(row):
    plan = row['plan_type']
    if plan == 'Premium' and row['amount'] != 12.99:
        return 12.99
    elif plan == 'Standard' and row['amount'] != 7.99:
        return 7.99
    elif plan == 'Student Plan' and row['amount'] not in [4.99, -5.99]:
        return 4.99
    return abs(row['amount'])

subscriptions['amount'] = subscriptions.apply(correct_amount, axis=1)

# Validate plan types
valid_plans = ['Premium', 'Standard', 'Student Plan']
subscriptions = subscriptions[subscriptions['plan_type'].isin(valid_plans)]

# Ensure churn_status aligns with renewal status
subscriptions['churn_status'] = subscriptions['renewed'].apply(lambda x: 'churned' if x == False else 'active')

# Validate date relationships
subscriptions['payment_date'] = pd.to_datetime(subscriptions['payment_date'])
subscriptions['valid_until'] = pd.to_datetime(subscriptions['valid_until'])

def validate_dates(row):
    if row['plan_type'] == 'Premium':
        expected_end = row['payment_date'] + pd.DateOffset(months=2)
    else:
        expected_end = row['payment_date'] + pd.DateOffset(months=1)
    return row['valid_until'] == expected_end

subscriptions = subscriptions[subscriptions.apply(validate_dates, axis=1)]

# Churn rate analysis
churn_analysis = subscriptions.groupby('plan_type').agg(
    total_users=('user_id', 'count'),
    churned_users=('churn_status', lambda x: (x == 'churned').sum())
).reset_index()

churn_analysis['churn_rate'] = round(
    (churn_analysis['churned_users'] / churn_analysis['total_users']) * 100, 2
)

subscriptions.to_csv("cleaned_subscriptions.csv", index=False)
churn_analysis.to_csv("subscription_analysis.csv", index=False)

print(churn_analysis)


      plan_type  total_users  churned_users  churn_rate
0       Premium            8              1       12.50
1      Standard          197             42       21.32
2  Student Plan          195             49       25.13


In [20]:
import pandas as pd

# Load data
ratings = pd.read_csv("ratings.csv")

# --- Data Cleaning ---
# 1. Remove duplicates
ratings = ratings.drop_duplicates(subset='rating_id')
ratings = ratings.drop_duplicates(subset=['user_id', 'movie_id'])

# 2. Fix invalid ratings
ratings['rating'] = ratings['rating'].clip(lower=0, upper=5)
ratings['rating'] = (ratings['rating'] * 2).round() / 2

# 3. Handle missing values
median_rating = ratings['rating'].median()
ratings['rating'] = ratings['rating'].fillna(median_rating)
most_common_date = ratings['review_date'].mode()[0]
ratings['review_date'] = ratings['review_date'].fillna(most_common_date)

# 4. Validate review dates
ratings['review_date'] = pd.to_datetime(ratings['review_date'], errors='coerce')
ratings = ratings[ratings['review_date'] >= pd.to_datetime('2024-11-01')]

# --- Analysis ---
movie_ratings = ratings.groupby('movie_id').agg(
    total_ratings=('rating', 'count'),
    average_rating=('rating', 'mean')
).reset_index()

user_stats = ratings.groupby('user_id').agg(
    ratings_given=('rating', 'count'),
    avg_rating=('rating', 'mean')
).reset_index()

# Save cleaned data & analysis
ratings.to_csv("cleaned_ratings.csv", index=False)
movie_ratings.to_csv("movie_ratings_analysis.csv", index=False)
user_stats.to_csv("user_behavior_analysis.csv", index=False)

print("Cleaning and analysis complete!")
print("\nTop 5 Movies by Rating:")
print(movie_ratings.sort_values(by='average_rating', ascending=False).head())


Cleaning and analysis complete!

Top 5 Movies by Rating:
      movie_id  total_ratings  average_rating
29          33              1             5.0
22          26              1             5.0
3118      4169              1             5.0
2085      2815              2             5.0
1534      2074              1             5.0


In [22]:
import pandas as pd

def clean_watch_history(watch_file, cleaned_users_file, output_file):
    watch_df = pd.read_csv(watch_file)
    users_df = pd.read_csv(cleaned_users_file)
    
    print("Original data shape:", watch_df.shape)
    
    initial_count = len(watch_df)
    watch_df = watch_df.drop_duplicates(subset='watch_id', keep='first')
    duplicates_removed = initial_count - len(watch_df)
    print(f"Removed {duplicates_removed} duplicate watch_id entries")
    
    valid_user_ids = users_df['user_id'].unique()
    valid_user_mask = watch_df['user_id'].isin(valid_user_ids)
    invalid_user_count = len(watch_df[~valid_user_mask])
    watch_df = watch_df[valid_user_mask].copy()
    print(f"Removed {invalid_user_count} entries with invalid user_ids")
    
    duration_mask = watch_df['watch_duration'].between(1, 240)
    invalid_duration = len(watch_df[~duration_mask])
    watch_df = watch_df[duration_mask].copy()
    print(f"Removed {invalid_duration} invalid watch durations")
    
    valid_devices = ['Laptop', 'Mobile', 'Smart TV']
    watch_df['device_type'] = (
        watch_df['device_type']
        .replace({'': 'Unknown'})
        .where(watch_df['device_type'].isin(valid_devices), 'Unknown')
    )
    invalid_device_count = len(watch_df[watch_df['device_type'] == 'Unknown'])
    print(f"Standardized {invalid_device_count} device entries to 'Unknown'")

    watch_df['watch_date'] = pd.to_datetime(watch_df['watch_date'], errors='coerce')
    date_mask = watch_df['watch_date'].notna()
    invalid_dates = len(watch_df[~date_mask])
    watch_df = watch_df[date_mask].copy()
    print(f"Removed {invalid_dates} invalid dates")

    watch_df = watch_df.dropna(subset=['watch_duration'])
    print(f"Final cleaned data shape: {watch_df.shape}")
    
    watch_df.to_csv(output_file, index=False)
    print(f"Cleaned data saved to {output_file}")

clean_watch_history(
    watch_file="watch_history.csv",
    cleaned_users_file="cleaned_users.csv",
    output_file="cleaned_watch_history.csv"
)


Original data shape: (98923, 6)
Removed 0 duplicate watch_id entries
Removed 2706 entries with invalid user_ids
Removed 725 invalid watch durations
Standardized 771 device entries to 'Unknown'
Removed 0 invalid dates
Final cleaned data shape: (95492, 6)
Cleaned data saved to cleaned_watch_history.csv


In [23]:
import pandas as pd

def clean_movies(input_path, output_path):
    df = pd.read_csv(input_path)

    df["release_year"] = df["release_year"].astype("Int64")
    df = df[df["runtime"].between(60, 240)]
    df["genre"] = df["genre"].str.title()
    df["title"] = df["title"].str.strip()

    df.to_csv(output_path, index=False)
    print(f"Cleaned data saved to {output_path}. Removed {len(df)} entries.")

if __name__ == "__main__":
    clean_movies("movies.csv", "cleaned_movies.csv")


Cleaned data saved to cleaned_movies.csv. Removed 4221 entries.
